## Mastering Machine Learning 2025

Taller 2: tokenizers y embeddings

Antes de iniciar abra este cuaderno en Google Colab y habilite la ejecución con GPU:
- En el menú Entorno de ejecución seleccione Cambiar tipo entorno de ejecución.
- Asegúrese de tener seleccionado Python 3.
- Como Acelerador de hardware seleccione GPU T4.

In [1]:
!pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 871.5 kB/s eta 0:00:28
   ---------------------------------------- 0.1/24.0 MB 1.1 MB/s eta 0:00:23
    --------------------------------------- 0.4/24.0 MB 2.5 MB/s eta 0:00:10
   - -------------------------------------- 1.0/24.0 MB 5.3 MB/s eta 0:00:05
   - -------------------------------------- 1.1/24.0 MB 4.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.7/24.0 MB 5.9 MB/s eta 0:00:04
   --- ------------------------------------ 2.0/24.0 MB 6.0 MB/s eta 0:00:04
   ---- ----------------------------------- 2.4/24.0 MB 6.8 MB/s eta 0:00:04
   ----- ---------------------------------- 3.1/24.0 MB 7.5 MB/s eta 0:00:03
   ----- ---------------------------------- 3.4/24.0 MB 8.0 MB/s eta 0:00:03
   ------ --------------------------------- 4.0/24.0 MB 8.3 MB/s eta 0:00:03
   ------- 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\ascas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Descarguemos un embedding entrenado con los datos de Wikipedia. El tamaño del vector del embedding es de 50 y tamaño de la descarga es cerca de 66MB.

Puede encontrar otras opciones en https://github.com/RaRe-Technologies/gensim-data

In [2]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [3]:
model.most_similar([model['song']], topn=11)

[('song', 1.0),
 ('album', 0.9297007918357849),
 ('songs', 0.90098637342453),
 ('soundtrack', 0.8414768576622009),
 ('albums', 0.8228148221969604),
 ('pop', 0.8219155669212341),
 ('sings', 0.8201141357421875),
 ('tune', 0.8188669681549072),
 ('duet', 0.8186635971069336),
 ('remix', 0.8086214065551758),
 ('band', 0.8063263297080994)]

## Word2Vec para generar embeddings

Word2Vec emplea contrastive learning, donde se comparan una palabra contra otra para determianr si pertenecen al mismo contexto o no. Se emplean ejemplos positivos (obtenidos del mismo texto) y negativos (obtenidos al azar).

Importamos las dependencias necesarias

In [ ]:
import pandas as pd
from urllib import request

Cargamos los datos con un conjunto de datos de playlists

In [ ]:
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

Saltamos las dos primeras líneas ya que no tienen datos útiles para el ejercicio (solo metadatos)

In [ ]:
lines = data.read().decode("utf-8").split('\n')[2:]

Eliminamos playlists con una sola canción dado que no sirven para generar información del contexto en el que aparecen canciones similares (misma playlist)

In [ ]:
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

Exploremos algunas de las playlists para ver su contenido (IDs de las canciones)

In [ ]:
print( f'Playlist #1:\n {playlists[0]}\n')
print( f'Playlist #2:\n {playlists[1]}')

Importamos el modelo Word2Vec y lo entrenamos con el dataset de playlists

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

Una vez entrenado, solocitamos canciones similares a una canción en particular

In [ ]:
song_id = 2172

model.wv.most_similar(positive=str(song_id))

Traemos y procesamos los datos de las canciones para tener acceso a sus títulos y artistas

In [ ]:
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

Traemos la información de la canción que seleccionamos

In [ ]:
print(songs_df.iloc[song_id])

Definimos una función para mostrar las canciones más similares a una seleccionada en formato texto

In [ ]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

Empleamos la función creada con la canción seleccionada

In [ ]:
print_recommendations(song_id)

In [ ]:
song_id = 2222
print(songs_df.iloc[song_id])
print_recommendations(song_id)